# Imports

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import TensorDataset, DataLoader

In [14]:
no_users = 50
seq_len = 4
var_1 = np.random.uniform(0,1,no_users*seq_len)
mu, sigma = 0, 0.1 # mean and standard deviation
s = np.random.normal(mu, sigma, seq_len)
corr_factors = 1+s

data = {'date': np.tile(np.array(['2021-03-01', '2021-03-02', '2021-03-03', '2021-03-04']), no_users), 
        'user_id': np.repeat(np.array([range(no_users)]),seq_len),
        'playtime': var_1,
        'logins': var_1 * np.repeat(corr_factors,no_users),
        'target': np.repeat(np.random.randint(2, size=no_users),seq_len)}
df = pd.DataFrame.from_dict(data)
df.head()

,date,user_id,playtime,logins,target
0,2021-03-01,0,0.206765,0.205667,1
1,2021-03-02,0,0.757943,0.753918,1
2,2021-03-03,0,0.204527,0.203441,1
3,2021-03-04,0,0.405417,0.403264,1
4,2021-03-01,1,0.858625,0.854065,0


In [16]:
features=['playtime', 'logins']

x = np.swapaxes(df[features].values.reshape(no_users, seq_len, -1),1,2)
x[:3]

array([[[0.20676549, 0.75794302, 0.20452662, 0.40541737],
        [0.20566748, 0.75391804, 0.2034405 , 0.40326444]],

       [[0.85862457, 0.98935219, 0.16736154, 0.30483113],
        [0.85406494, 0.98409834, 0.16647278, 0.30321235]],

       [[0.05357769, 0.56437058, 0.50396775, 0.73634384],
        [0.05329317, 0.56137355, 0.50129148, 0.73243357]]])

In [17]:
y = df.groupby('user_id').target.max().values
y[:3]

array([1, 0, 1])

In [18]:
x.shape, y.shape

((50, 2, 4), (50,))

## to dataset and dataloader

In [19]:
def create_datasets(data, target, train_size, valid_pct=0.1, seed=None):
    """Converts NumPy arrays into PyTorch datsets."""
    
    sz = train_size
    idx = np.arange(sz)
    trn_idx, val_idx = train_test_split(
        idx, test_size=valid_pct, random_state=seed)
    trn_ds = TensorDataset(
        torch.tensor(data[:sz][trn_idx]).float(), 
        torch.tensor(target[:sz][trn_idx]).long())
    val_ds = TensorDataset(
        torch.tensor(data[:sz][val_idx]).float(), 
        torch.tensor(target[:sz][val_idx]).long())
    tst_ds = TensorDataset(
        torch.tensor(data[sz:]).float(),  
        torch.tensor(target[sz:]).long())
    return trn_ds, val_ds, tst_ds

In [20]:
def create_loaders(data, bs=128, jobs=0):
    """Wraps the datasets returned by create_datasets function with data loaders."""
    
    trn_ds, val_ds, tst_ds = data
    trn_dl = DataLoader(trn_ds, batch_size=bs, shuffle=True, num_workers=jobs)
    val_dl = DataLoader(val_ds, batch_size=bs, shuffle=False, num_workers=jobs)
    tst_dl = DataLoader(tst_ds, batch_size=bs, shuffle=False, num_workers=jobs)
    return trn_dl, val_dl, tst_dl

In [23]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
trn_sz=40
datasets = create_datasets(x, y, trn_sz, seed=1234)
trn_dl, val_dl, tst_dl = create_loaders(datasets, bs=256)

# Define Model

In [ ]:
class _SepConv1d(nn.Module):
    """A simple separable convolution implementation.
    
    The separable convlution is a method to reduce number of the parameters 
    in the deep learning network for slight decrease in predictions quality.
    """
    def __init__(self, ni, no, kernel, stride, pad):
        super().__init__()
        self.depthwise = nn.Conv1d(ni, ni, kernel, stride, padding=pad, groups=ni)
        self.pointwise = nn.Conv1d(ni, no, kernel_size=1)

    def forward(self, x):
        return self.pointwise(self.depthwise(x))

In [ ]:
class SepConv1d(nn.Module):
    """Implementes a 1-d convolution with 'batteries included'.
    
    The module adds (optionally) activation function and dropout layers right after
    a separable convolution layer.
    """
    def __init__(self, ni, no, kernel, stride, pad, drop=None,
                 activ=lambda: nn.ReLU(inplace=True)):
    
        super().__init__()
        assert drop is None or (0.0 < drop < 1.0)
        layers = [_SepConv1d(ni, no, kernel, stride, pad)]
        if activ:
            layers.append(activ())
        if drop is not None:
            layers.append(nn.Dropout(drop))
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x): 
        return self.layers(x)

In [ ]:
class Flatten(nn.Module):
    """Converts N-dimensional tensor into 'flat' one."""

    def __init__(self, keep_batch_dim=True):
        super().__init__()
        self.keep_batch_dim = keep_batch_dim

    def forward(self, x):
        if self.keep_batch_dim:
            return x.view(x.size(0), -1)
        return x.view(-1)

In [ ]:
class Classifier(nn.Module):
    def __init__(self, raw_ni, no, drop=.5):
        super().__init__()
        
        # no_input, no_output, kernel, stride, padding
        # mit kernel=2 wird in diesem Fall immer eine seq_len gekillt, was dazu führt, dass nach 2 Convolutions keine Conv1d mehr möglich ist. Das später anpassen
        self.raw = nn.Sequential(
            SepConv1d(raw_ni,  32, 2, 1, 0, drop=drop),
            SepConv1d(    32,  64, 2, 1, 0, drop=drop),
            SepConv1d(    64,  256, 2, 2, 0, drop=drop),
            # SepConv1d(    64, 128, 2, 1, 0, drop=drop),
            # SepConv1d(   128, 256, 2, 1, 0),
            Flatten(),
            nn.Dropout(drop), nn.Linear(256, 64), nn.ReLU(inplace=True),
            nn.Dropout(drop), nn.Linear( 64, 64), nn.ReLU(inplace=True))
        
        self.out = nn.Sequential(
            nn.Linear(64, 64), nn.ReLU(inplace=True), nn.Linear(64, no))
        
    def forward(self, t_raw):
        raw_out = self.raw(t_raw)
        out = self.out(raw_out)
        return out

In [ ]:
raw_feat = x.shape[1]

lr = 0.001
n_epochs = 1000
iterations_per_epoch = len(trn_dl)
num_classes = 2
best_acc = 0
patience, trials = 1500, 0
base = 1
step = 2
loss_history = []
acc_history = []

model = Classifier(raw_feat, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(), lr=lr)

In [ ]:
for epoch in range(1):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(trn_dl):
        x_raw, y_batch = [t.to(device) for t in batch]
x_raw.shape

In [ ]:
raw_ni=x.shape[1] # input feature
drop=0.3
m = SepConv1d(raw_ni,  32, 2, 1, 0, drop=drop)
output_ = m(x_raw)
m = SepConv1d(32,  64, 2, 1, 0, drop=drop)
output_ = m(output_)
m = SepConv1d(64,  256, 2, 2, 0, drop=drop)
output_ = m(output_)

In [ ]:
output_.shape

In [ ]:
trn_sz = 5

In [ ]:
print('Start model training')

for epoch in range(1, n_epochs + 1):
    
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(trn_dl):
        x_raw, y_batch = [t.to(device) for t in batch]
        opt.zero_grad()
        out = model(x_raw)
        loss = criterion(out, y_batch)
        epoch_loss += loss.item()
        loss.backward()
        opt.step()
        
    epoch_loss /= trn_sz
    loss_history.append(epoch_loss)
    
    model.eval()
    correct, total = 0, 0
    for batch in trn_dl:  # muss noch auf val_loader angepasst werden
        x_raw, y_batch = [t.to(device) for t in batch]
        out = model(x_raw)
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        total += y_batch.size(0)
        correct += (preds == y_batch).sum().item()
    
    acc = correct / total
    acc_history.append(acc)

    if epoch % base == 0:
        print(f'Epoch: {epoch:3d}. Loss: {epoch_loss:.4f}. Acc.: {acc:2.2%}')
        base *= step

    if acc > best_acc:
        trials = 0
        best_acc = acc
        torch.save(model.state_dict(), 'best.pth')
        print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
    else:
        trials += 1
        if trials >= patience:
            print(f'Early stopping on epoch {epoch}')
            break
            
print('Done!')

In [ ]:
def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(12, 4))

ax[0].plot(loss_history, label='loss')
ax[0].set_title('Validation Loss History')
ax[0].set_xlabel('Epoch no.')
ax[0].set_ylabel('Loss')

ax[1].plot(smooth(acc_history, 5)[:-2], label='acc')
ax[1].set_title('Validation Accuracy History')
ax[1].set_xlabel('Epoch no.')
ax[1].set_ylabel('Accuracy');